In [1]:
import logging
from typing import Dict, List, Optional, Union
from pprint import pprint
import pandas as pd

from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor, EmbeddingRetriever, DensePassageRetriever
from haystack.utils import convert_files_to_docs, print_answers
from haystack.document_stores import InMemoryDocumentStore, FAISSDocumentStore
from haystack.nodes import FARMReader, TransformersReader, RAGenerator, Seq2SeqGenerator
from haystack.pipelines import GenerativeQAPipeline
from haystack.schema import Document

import torch
from transformers import PreTrainedTokenizer, BatchEncoding
import datasets

%load_ext autoreload
%autoreload 2

In [2]:
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# GET documents

In [3]:
# eli5 = datasets.load_dataset('eli5')

# wikitext = datasets.load_dataset('wikitext','wikitext-2-raw-v1')

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# wikitext['train']['text'][:30]

In [3]:
# len(wikitext['train'])

In [3]:
DATA_PATH = '../data/bitcoin_articles.csv'

df = pd.read_csv(DATA_PATH)
df.shape

(2500, 17)

In [4]:
df

,article_id,title,author,published_date,link,clean_url,excerpt,summary,rights,article_rank,topic,country,language,authors,media,twitter_account,article_score
0,57a00c1140cbd3af79e77bf0e4e6af48,62% of Bitcoin Has Not Moved in a Year as Long-Term Holders Refuse to Sell,Jamie McNeill,04-10-2022 17:15,https://www.business2community.com/crypto-news/62-of-bitcoin-has-not-moved-i...,business2community.com,"Over the course of the last few years, there has been an impressive trend wh...","Over the course of the last few years, there has been an impressive trend wh...",business2community.com,1595,finance,US,en,Jamie McNeill,https://www.business2community.com/wp-content/uploads/2022/10/btcc.webp,@Jamie_DeFi,8.556426
1,21b48b3731c03466be3fac4be6c7dc67,The Orange Party Issue Playlist,Bitcoin Magazine,05-10-2022 21:17,https://bitcoinmagazine.com/culture/orange-party-issue-playlist,bitcoinmagazine.com,News Links: Russia Legalizing Bitcoin And Crypto Is A Matter Of Time Says Mi...,Russia Legalizing Bitcoin And Crypto Is A Matter Of Time Says Minister Of In...,bitcoinmagazine.com,6284,news,US,en,Bitcoin Magazine,https://bitcoinmagazine.com/.image/t_share/MTkyODIyNDQ0Mjc5NDczMzcz/playlist...,NaN,8.507881
2,77030740ee160ad68c25e4e63515dd77,How Many Bitcoins Are There?,AOL Staff,04-10-2022 21:44,https://www.gobankingrates.com/investing/crypto/how-many-bitcoins-are-there/,gobankingrates.com,"Bitcoin has a maximum supply of 21 million. However, that doesn't tell the w...","Bitcoin has a maximum supply of 21 million. However, that doesn't tell the w...",aol.com,5044,news,US,en,"AOL Staff,David Granahan",https://s.yimg.com/ny/api/res/1.2/wPK4V8gjwjrDYoMuUGxoBw--/YXBwaWQ9aGlnaGxhb...,@AOL,8.483973
3,19285dde79599b6a9b4aa5a58805159b,Bitcoin 2008–2022: From Bouncing Baby To Troubled Teenager,Mark Hooson,03-10-2022 15:45,https://www.forbes.com/advisor/au/investing/bitcoin-2008-2022-from-bouncing-...,forbes.com,"Conceived in 2008 and launched in 2009, bitcoin is the world's first and lar...","Conceived in 2008 and launched in 2009, bitcoin is the world's first and lar...",forbes.com,49,business,US,en,"Forbes Staff,Kevin Pratt,Mark Hooson,Sophie Venz",https://thumbor.forbes.com/thumbor/fit-in/x/https://www.forbes.com/advisor/a...,NaN,8.466393
4,ad1822c7ff1bbd7e38f7f9dbdbce245f,"Bitcoin Lightning Network capacity strikes 5,000 BTC",Joseph Hall,03-10-2022 12:09,https://cointelegraph.com/news/bitcoin-lightning-network-capacity-strikes-5-...,cointelegraph.com,"First created in 2018, the Lighting Network has come under fire recently, wi...",Bear markets are for building out capacity on the layer-2 Lightning Network....,cointelegraph.com,1696,news,US,en,Joseph Hall,https://images.cointelegraph.com/images/1200_aHR0cHM6Ly9zMy5jb2ludGVsZWdyYXB...,@cointelegraph,8.446906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2e00dee283798fb5be8339146ddcf9f7,"Banks Around the World Have Over $9,200,000,000 in Crypto Asset Exposure: Ne...",NaN,03-10-2022 18:41,https://dailyhodl.com/2022/10/03/banks-around-the-world-have-over-9200000000...,dailyhodl.com,New research published by an international committee formed to develop banki...,New research published by an international committee formed to develop banki...,dailyhodl.com,20705,news,US,en,NaN,https://dailyhodl.com/wp-content/uploads/2022/10/banks-around-asset-exposure...,@thedailyhodl,6.743612
2496,d198a4ed4d020fc72dd1ecf3be2780d1,Crypto Was the Best Performing Asset in Q3 2022 – October Could Be Even Better,Ali B,04-10-2022 22:12,https://www.business2community.com/crypto-news/crypto-was-the-best-performin...,business2community.com,"Crypto was the best performing asset in Q3, 2022, and the cryptocurrency pri...","Crypto was the best performing asset in Q3, 2022, and the cryptocurrency pri...",business2community.com,1595,finance,US,en,Ali B,https://www.business2community.com/wp-content/uploads/2022/10/Screenshot_6-1...,@b2community,6.743447
2497,632f19b634b80b0882878c21771f36a1,Edu Gov Footer Bac

In [5]:
# Use data to initialize Document objects
ids = list(df["article_id"].values)
texts = list(df["summary"].values)
titles = list(df["title"].values)
dates = list(df["published_date"].values)
links = list(df["link"].values)

all_docs = []
for i, title, text, date, link in zip(ids, titles, texts, dates, links):
    all_docs.append(Document(id=i, 
                             content=text, 
                             meta={"article_title": title or "", "article_link": link, "article_date": date}))


# all_docs = [
#     {
#         'id':i, 
#         'content': c,
#         'meta': {'article_link': l, 'article_title': t, 'article_date': d}
#     } for i, c, t, d, l in zip(df['article_id'], df['summary'], df['title'], df['published_date'], df['link'])
# ]

In [6]:
len(all_docs)

2500

In [ ]:
# PDFS_PATH="/data/kg_pdfs_test/"
# all_docs = convert_files_to_docs(dir_path=PDFS_PATH)

In [7]:
torch.__version__

'1.12.1+cu113'

In [8]:
torch.version.cuda

'11.3'

## Preprocessing 

In [7]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=128,  # smaller splits works better? 
    split_respect_sentence_boundary=True,
)

all_docs_process = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(all_docs_process)}")

Preprocessing:   0%|          | 0/2500 [00:00<?, ?docs/s]

n_files_input: 2500
n_docs_output: 2507


In [8]:
all_docs_process[2:5]

[<Document: {'content': "Bitcoin has a maximum supply of 21 million. However, that doesn't tell the whole story. As time goes on, it becomes increasingly difficult to acquire bitcoin due to the asset's economics. Here is a full breakdown of how bitcoin works and exactly why it has a limited supply.Read: Looking To Diversify In A Bear Market? Consider These 6 Alternative InvestmentsBitcoin Tokenomics: Proof of Work, Mining and Halving CyclesTokenomics, as one can imagine, refers to the economics of a token. Similar to how fiat currencies, like the dollar, are issued by governments and regulated through monetary policy, tokenomics refers to the rules and functions revolving around cryptocurrency.", 'content_type': 'text', 'score': None, 'meta': {'article_title': 'How Many Bitcoins Are There?', 'article_link': 'https://www.gobankingrates.com/investing/crypto/how-many-bitcoins-are-there/', 'article_date': '04-10-2022 21:44', '_split_id': 0}, 'embedding': None, 'id': '7afbcf9ee51ca07473ee35

## Document Store 

In [9]:
# In-Memory Document Store
# document_store = InMemoryDocumentStore()


# The FAISSDocumentStore uses a SQL(SQLite in-memory be default) database under-the-hood to store the document text and other meta data. 
# The vector embeddings of the text are indexed on a FAISS Index that later is queried for searching answers.
document_store = FAISSDocumentStore(sql_url = "sqlite:///faiss_document_store.db", 
                                    faiss_index_factory_str="Flat", similarity="dot_product", return_embedding=True)

In [10]:
document_store.write_documents(all_docs_process)

Writing Documents:   0%|          | 0/2507 [00:00<?, ?it/s]

In [11]:
document_store.get_document_count()

1938

In [12]:
document_store.get_all_documents()[55]

<Document: {'content': "Paxful, the global peer-to-peer fintech platform, has been announced as one of the lead sponsors for the first annual Africa Bitcoin Conference (ABC), taking place on 5 – 7 December, 2022 in Accra, Ghana. \xa0 The conference, titled; 'Accelerate Bitcoin Adoption in Africa,' aims to foster collaboration among thousands of Bitcoin developers, technologists, investors, advocates, and educators working in the African bitcoin ecosystem. Additionally, the event hopes to strengthen knowledge and resource sharing and form connections between African influencers and those working around the globe.", 'content_type': 'text', 'score': None, 'meta': {'article_title': 'Paxful Becomes Africa Bitcoin Conference Major Sponsor', 'article_link': 'https://www.thisdaylive.com/index.php/2022/10/07/paxful-becomes-africa-bitcoin-conference-major-sponsor/', 'article_date': '07-10-2022 00:00', '_split_id': '0'}, 'embedding': None, 'id': '1588e1a5ec55e8a167ec8e9550809461'}>

## Retriever


In [ ]:
# Initialize DPR Retriever to encode documents, encode question and query documents

dpr_retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=False,
    embed_title=True,
)

In [14]:
# Add / update documents embeddings to index

# document_store.update_embeddings(retriever=dpr_retriever)

INFO:haystack.document_stores.faiss:Updating embeddings for 1938 docs...


Updating Embedding:   0%|          | 0/1938 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/1952 [00:00<?, ? Docs/s]

# CHECKPOINT 

In [3]:
# document_store.save("my_faiss_index.faiss")

document_store = FAISSDocumentStore.load("my_faiss_index.faiss","my_faiss_index.json")

## Retriever


In [4]:
# Initialize DPR Retriever to encode documents, encode question and query documents

dpr_retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=False,
    embed_title=True,
)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


In [6]:
document_store.get_all_documents()[55] #.embedding.shape

<Document: {'content': "Paxful, the global peer-to-peer fintech platform, has been announced as one of the lead sponsors for the first annual Africa Bitcoin Conference (ABC), taking place on 5 – 7 December, 2022 in Accra, Ghana. \xa0 The conference, titled; 'Accelerate Bitcoin Adoption in Africa,' aims to foster collaboration among thousands of Bitcoin developers, technologists, investors, advocates, and educators working in the African bitcoin ecosystem. Additionally, the event hopes to strengthen knowledge and resource sharing and form connections between African influencers and those working around the globe.", 'content_type': 'text', 'score': None, 'meta': {'vector_id': '55', 'article_title': 'Paxful Becomes Africa Bitcoin Conference Major Sponsor', 'article_link': 'https://www.thisdaylive.com/index.php/2022/10/07/paxful-becomes-africa-bitcoin-conference-major-sponsor/', 'article_date': '07-10-2022 00:00', '_split_id': '0'}, 'embedding': '<embedding of shape (768,)>', 'id': '1588e1

In [5]:
query = "When was bitcoin created?"
query1 = "How to use the lightining network?"
query2 = "What is bitcoin mining?"

In [6]:
dpr_ls = [(doc.content, doc.meta) for doc in dpr_retriever.retrieve(query2, top_k=5)]

pprint(dpr_ls)

[('Bitcoin has come a long way since its inception in 2008, becoming the most '
  'popular and valued cryptocurrency of all time. But with time, its '
  'inefficiencies were also exposed -- lack of consumer protection leading to '
  'money laundering and black market trading. Perhaps the most concerning '
  'issue is the huge impact its production has on the environment.\n'
  '\n'
  'Mining Bitcoins could be worse than extracting gold and is as bad as '
  "industries related to cattle farming and crude oil. At least that's what "
  'the latest study published in the journal Scientific Reports suggests.',
  {'_split_id': '0',
   'article_date': '04-10-2022 05:53',
   'article_link': 'https://www.gizbot.com/amphtml/internet/features/bitcoin-mining-might-be-as-grave-as-oil-drilling-for-earth-082900.html',
   'article_title': 'Bitcoin Mining Might Be As Grave As Oil Drilling For '
                    'Earth',
   'vector_id': '516'}),
 ('Bitcoin mining is the process used to generate new co

# Generator

## Retrieval Augmented Generator

In [ ]:
# Initialize RAG Generator

rag_generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=100,
    min_length=2,
    embed_title=True,
    num_beams=2,
)

In [ ]:
Q = query2

ans = rag_generator.predict(Q, 
                            documents=dpr_retriever.retrieve(Q, top_k=5), 
                            top_k=3)

# pprint(ans.get('answers'))
pprint(ans)

## T5-large

In [7]:
class _T5Converter:
    """
    A sequence-to-sequence model input converter (https://huggingface.co/yjernite/bart_eli5) based on the BART architecture fine-tuned on ELI5 dataset (https://arxiv.org/abs/1907.09190).
    The converter takes documents and a query as input and formats them into a single sequence that a seq2seq model can use it as input for its generation step.
    This includes model-specific prefixes, separation tokens and the actual conversion into tensors. 
    For more details refer to Yacine Jernite's excellent LFQA contributions at https://yjernite.github.io/lfqa.html
    """
    def __call__(self, tokenizer: PreTrainedTokenizer, query: str, documents: List[Document], top_k: Optional[int] = None) -> BatchEncoding:
        conditioned_doc = "<P> " + " <P> ".join([d.content for d in documents])
        # print(conditioned_doc)

        # concatenate question and support document into BART input
        query_and_docs = "question: {} context: {}".format(query, conditioned_doc)
        max_source_length = 512

        # return tokenizer([(query_and_docs, "A")], truncation=True, padding=True, max_length=max_source_length, return_tensors="pt")
        return tokenizer([query_and_docs], truncation=True, padding=True, max_length=max_source_length, return_tensors="pt")

In [8]:
# /data/t5-large; google/t5-large-lm-adapt

t5_generator = Seq2SeqGenerator(
    model_name_or_path="t5-small",
    input_converter=_T5Converter(),
    use_gpu=True,
    top_k=1,
    max_length=100,
    min_length=2,
    num_beams=2,
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/home/jerm-xps/venv_qna/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
Q = query2

ans = t5_generator.predict(Q, 
                           documents=dpr_retriever.retrieve(Q, top_k=5), 
                           top_k=3)

# pprint(ans.get('answers'))
pprint(ans)

{'answers': [<Answer {'answer': 'the process used to generate new coins and verify new transactions', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['5323100ade27d773eec3ebcc4b65521', 'da3d945c6fa1283e0701d984dce654ad', 'c5cc43a36291e5923c46b7e83780336a', '7411498bc20c0fc50fd5550b0282acf5', '10133cab79a62fdb758453c62bd05d5b'], 'doc_scores': [0.6976991824052634, 0.6934953021726374, 0.6927373812673804, 0.6916624142932156, 0.6916624142932156], 'content': ["Bitcoin has come a long way since its inception in 2008, becoming the most popular and valued cryptocurrency of all time. But with time, its inefficiencies were also exposed -- lack of consumer protection leading to money laundering and black market trading. Perhaps the most concerning issue is the huge impact its production has on the environment.\n\nMining Bitcoins could be worse than extracting gold and is as bad as industries re

## Pipeline

In [23]:
QUESTIONS = [
    "When was bitcoin created?",
    "How do I use the lightining network?",
    "What is bitcoin mining?",
    "What is the difference between bitcoin and ethereum?",
    "How to buy bitcoin?",
    "What are bitcoin whales?",
    "What was bitcoin's price in 2020?",
    "How much was bitcoin's price in 2021?",
    "What is the market cap of btc?"
]

In [24]:
pipe_GQA = GenerativeQAPipeline(generator=t5_generator, retriever=dpr_retriever)

for question in QUESTIONS:
    res = pipe_GQA.run(query=question, 
                       params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 5}})
    
    (print_answers(res, details="all"))


Query: When was bitcoin created?
Answers:
[   <Answer {'answer': '2009', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['de2e2bc7d6532714d5ad2334942149c9', '3107a00846fef491728266abbabb5c59', 'a25cc76071b03efed19fe3535373d9a9', '49d82e74e302615bdbb8ee19a9b35291', 'edd3c9c1fd287eb22d2a730539d82d42'], 'doc_scores': [0.6950108059107912, 0.6878239914940308, 0.6878239914940308, 0.6878239914940308, 0.6874300817937259], 'content': ['Released on April 31st, 2009, Bitcoin is a cryptocurrency (a.k.a. digital money) that allows users to facilitate pseudonymous transactions over the internet. Unlike Fiat money, Bitcoin is decentralized, which means that it is not regulated by a central governing body. Bitcoin users can, therefore, facilitate peer-to-peer cash transactions without the approval of any financial institution.\nGamers enjoy numerous benefits by using Bitcoin. For instance, this cr